### Importing dependencies

In [1]:
# Importing various libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from PIL import Image
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [ ]:
# From path of image directory making list of image path
train = '/content/drive/MyDrive/Data Science Learning/Chest X-Ray Images (Pneumonia)/train'
val = '/content/drive/MyDrive/Data Science Learning/Chest X-Ray Images (Pneumonia)/val'

# Doing preprocessing of image data

In [ ]:
# Image parameters
img_size = 224
batch_size = 32

In [ ]:
# Image data generator
data_gen = ImageDataGenerator(
    rescale = 1./255,
)

In [ ]:
# Train generator
train_gen = data_gen.flow_from_directory(
    train,
    target_size = (img_size, img_size),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 5230 images belonging to 2 classes.


In [ ]:
# Val generator
val_gen = data_gen.flow_from_directory(
    val,
    target_size = (img_size, img_size),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 16 images belonging to 2 classes.


## Building CNN model

In [ ]:
# Building CNN architecture

model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())

model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(train_gen.num_classes, activation='sigmoid'))

In [ ]:
# Model Summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [ ]:
# Compile the model
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Training the model
history = model.fit(train_gen, epochs=5, validation_data=val_gen)     # Keeping only 5 epochs because it takes too much time

Epoch 1/5
164/164 [==============================] - 923s 6s/step - loss: 0.2989 - accuracy: 0.8836 - val_loss: 0.3918 - val_accuracy: 0.7500
Epoch 2/5
164/164 [==============================] - 80s 484ms/step - loss: 0.1234 - accuracy: 0.9551 - val_loss: 0.1189 - val_accuracy: 1.0000
Epoch 3/5
164/164 [==============================] - 81s 490ms/step - loss: 0.0758 - accuracy: 0.9723 - val_loss: 0.5618 - val_accuracy: 0.8125
Epoch 4/5
164/164 [==============================] - 80s 483ms/step - loss: 0.0596 - accuracy: 0.9784 - val_loss: 0.2144 - val_accuracy: 0.8750
Epoch 5/5
164/164 [==============================] - 80s 485ms/step - loss: 0.0466 - accuracy: 0.9824 - val_loss: 0.1797 - val_accuracy: 0.9375


By training the data .... Our model gives a validation score of 93.75%

### Building a predictive system

In [14]:
image_path = 'Enter image path'

In [10]:
# function to preprocess input image
def preprocess(image_path):
  img = cv2.imread(image_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (224,224))
  img = img/255
  img = np.reshape(img, [1,224,224,3])
  return img

In [11]:
# function to predict the class
def predict_image_class(model, image_path, class_indices):
    image = preprocess(image_path)
    prediction = model.predict(image)
    predicted_class_index = np.argmax(prediction)
    predicted_class_name = class_indices[predicted_class_index]
    return predicted_class_name

In [12]:
# create a mapping from class indices to class names
class_indices = {0: 'Normal', 1: 'Pneumonia'}

In [15]:
print(predict_image_class(model, image_path, class_indices))

1/1 [==============================] - 0s 50ms/step
Pneumonia
